##### Profile memory usage

- Using smaller numeric types
- Using categorical type instead of object (strings)
- Using DateTime

##### 1. Import libraries

In [1]:
import pandas as pd
import numpy as np

##### 2. Load the data by pandas

In [2]:
%%time
df = pd.read_csv('datasets/sale_data.csv')
df.head()

CPU times: total: 6.11 s
Wall time: 7.51 s


,Unnamed: 0,Region,Country,Item Type,Sales Channel,Order Priority,Order Date,Order ID,Ship Date,Units Sold,Unit Price,Unit Cost,Total Revenue,Total Cost,Total Profit
0,0,Sub-Saharan Africa,South Africa,Fruits,Offline,M,7/27/2012,443368995,7/28/2012,1593,9.33,6.92,14862.69,11023.56,3839.13
1,1,Sub-Saharan Africa,South Africa,Fruits,Offline,M,7/27/2012,443368995,7/28/2012,1593,9.33,6.92,14862.69,11023.56,3839.13
2,2,Middle East and North Africa,Morocco,Clothes,Online,M,9/14/2013,667593514,10/19/2013,4611,109.28,35.84,503890.08,165258.24,338631.84
3,3,Middle East and North Africa,Morocco,Clothes,Online,M,9/14/2013,667593514,10/19/2013,4611,109.28,35.84,503890.08,165258.24,338631.84
4,4,Australia and Oceania,Papua New Guinea,Meat,Offline,M,5/15/2015,940995585,6/4/2015,360,421.89,364.69,151880.40,131288.40,20592.00


##### 3.Initial Memory Usage

In [3]:
# initial usage, no optimization. see column dtypes
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500014 entries, 0 to 1500013
Data columns (total 15 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   Unnamed: 0      1500014 non-null  int64  
 1   Region          1500012 non-null  object 
 2   Country         1500008 non-null  object 
 3   Item Type       1500013 non-null  object 
 4   Sales Channel   1500013 non-null  object 
 5   Order Priority  1500014 non-null  object 
 6   Order Date      1500014 non-null  object 
 7   Order ID        1500014 non-null  int64  
 8   Ship Date       1500014 non-null  object 
 9   Units Sold      1500014 non-null  int64  
 10  Unit Price      1500014 non-null  float64
 11  Unit Cost       1500014 non-null  float64
 12  Total Revenue   1500009 non-null  float64
 13  Total Cost      1500009 non-null  float64
 14  Total Profit    1500009 non-null  float64
dtypes: float64(5), int64(3), object(7)
memory usage: 746.3 MB


In [4]:
# Total GiB (same as above)
start_memory = df.memory_usage(index=False, deep=True).sum() / (2 ** 30)
print(f"{round(start_memory, 3)} GiB")

0.729 GiB


##### 4. Specific Memory Profiling in column

In [5]:
memory_usage = df.memory_usage(deep=True) / 1024 ** 2

# Print the sum of memory usage for each column
memory_usage.head(15)

Index               0.000122
Unnamed: 0         11.444199
Region            104.203013
Country            94.273987
Item Type          93.818047
Sales Channel      90.838334
Order Priority     82.970440
Order Date         94.324372
Order ID           11.444199
Ship Date          94.324188
Units Sold         11.444199
Unit Price         11.444199
Unit Cost          11.444199
Total Revenue      11.444199
Total Cost         11.444199
dtype: float64

In [6]:
# All use memory_profiler
memory_usage.sum()

746.3060922622681

In [7]:
df = pd.read_csv("datasets/sale_data.csv")

# ทดสอบ drop บางคอลัมน์ที่ไม่ได้ใช้งานออกไป
df.drop(df[["Unnamed: 0","Region"]], axis=1, inplace=True)
df.head()

,Country,Item Type,Sales Channel,Order Priority,Order Date,Order ID,Ship Date,Units Sold,Unit Price,Unit Cost,Total Revenue,Total Cost,Total Profit
0,South Africa,Fruits,Offline,M,7/27/2012,443368995,7/28/2012,1593,9.33,6.92,14862.69,11023.56,3839.13
1,South Africa,Fruits,Offline,M,7/27/2012,443368995,7/28/2012,1593,9.33,6.92,14862.69,11023.56,3839.13
2,Morocco,Clothes,Online,M,9/14/2013,667593514,10/19/2013,4611,109.28,35.84,503890.08,165258.24,338631.84
3,Morocco,Clothes,Online,M,9/14/2013,667593514,10/19/2013,4611,109.28,35.84,503890.08,165258.24,338631.84
4,Papua New Guinea,Meat,Offline,M,5/15/2015,940995585,6/4/2015,360,421.89,364.69,151880.40,131288.40,20592.00


In [8]:
memory_usage = df.memory_usage(deep=True) / 1024 ** 2
memory_usage.sum()

630.6588802337646

##### 5.Use smaller numeric types

In [9]:
# Max int values (we have no negative numbers in our data)
print(np.iinfo(np.int64).max)
print(np.iinfo(np.int32).max)
print(np.iinfo(np.int16).max)
print(np.iinfo(np.int8).max)

9223372036854775807
2147483647
32767
127


In [10]:
# What is the max for each numeric column?
# print(df.dtypes)
print(df['Units Sold'].max())
print(df['Unit Price'].max())
print(df['Unit Cost'].max())
print(df['Total Revenue'].max())
print(df['Total Cost'].max())
print(df['Total Profit'].max())
print(df['Total Profit'].min())
print(df['Total Profit'].mean())

df.dtypes

10000
668.27
524.96
6682700.0
5249600.0
87500000000000.0
2.41
59392046.10664549


Country            object
Item Type          object
Sales Channel      object
Order Priority     object
Order Date         object
Order ID            int64
Ship Date          object
Units Sold          int64
Unit Price        float64
Unit Cost         float64
Total Revenue     float64
Total Cost        float64
Total Profit      float64
dtype: object

In [11]:
# Change Type of column
# df['column_name'] = df['column_name'].astype('int')
df['Order ID'] = df['Order ID'].astype('int32')
df['Units Sold'] = df['Units Sold'].astype('int32')

df['Unit Price'] = df['Unit Price'].astype('float32')
df['Unit Cost'] = df['Unit Cost'].astype('float32')
df['Total Revenue'] = df['Total Revenue'].astype('float32')
df['Total Cost'] = df['Total Cost'].astype('float32')
df['Total Profit'] = df['Total Profit'].astype('float32')

df.dtypes

Country            object
Item Type          object
Sales Channel      object
Order Priority     object
Order Date         object
Order ID            int32
Ship Date          object
Units Sold          int32
Unit Price        float32
Unit Cost         float32
Total Revenue     float32
Total Cost        float32
Total Profit      float32
dtype: object

In [12]:
memory_usage = df.memory_usage(deep=True) / 1024 ** 2
memory_usage.sum()

590.6041851043701

In [13]:
df

,Country,Item Type,Sales Channel,Order Priority,Order Date,Order ID,Ship Date,Units Sold,Unit Price,Unit Cost,Total Revenue,Total Cost,Total Profit
0,South Africa,Fruits,Offline,M,7/27/2012,443368995,7/28/2012,1593,9.330000,6.920000,1.486269e+04,1.102356e+04,3.839130e+03
1,South Africa,Fruits,Offline,M,7/27/2012,443368995,7/28/2012,1593,9.330000,6.920000,1.486269e+04,1.102356e+04,3.839130e+03
2,Morocco,Clothes,Online,M,9/14/2013,667593514,10/19/2013,4611,109.279999,35.840000,5.038901e+05,1.652582e+05,3.386318e+05
3,Morocco,Clothes,Online,M,9/14/2013,667593514,10/19/2013,4611,109.279999,35.840000,5.038901e+05,1.652582e+05,3.386318e+05
4,Papua New Guinea,Meat,Offline,M,5/15/2015,940995585,6/4/2015,360,421.890015,364.690002,1.518804e+05,1.312884e+05,2.059200e+04
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1500009,India,Snacks,Online,L,10/10/2012,440306556,11/20/2012,5349,152.580002,97.440002,NaN,NaN,NaN
1500010,NaN,Household,Online,L,12/5/2016,937431466,12/8/2016,5657,668.270020,502.540009,3.780404e+06,2.842869e+06,9.375346e+05
1500011,NaN,Household,Offline,C,11/19/2011,274930989,12/13/2011,7044,668.270020,502.540009,4.707294e+06,3.539892e+06,1.167402e+06
1500012,Brunei,Cereal,Offline,L,5/28/2013,153842341,7/16/2013,4222,205.699997,117.110001,NaN,NaN,NaN


##### 6. Use categorical type

In [14]:
df["Sales Channel"] = df["Sales Channel"].astype("category")
df["Item Type"] = df["Item Type"].astype("category")
df["Order Priority"] = df["Order Priority"].astype("category")

memory_usage = df.memory_usage(deep=True) / 1024 ** 2
memory_usage.sum()

327.27058506011963

In [15]:
df

,Country,Item Type,Sales Channel,Order Priority,Order Date,Order ID,Ship Date,Units Sold,Unit Price,Unit Cost,Total Revenue,Total Cost,Total Profit
0,South Africa,Fruits,Offline,M,7/27/2012,443368995,7/28/2012,1593,9.330000,6.920000,1.486269e+04,1.102356e+04,3.839130e+03
1,South Africa,Fruits,Offline,M,7/27/2012,443368995,7/28/2012,1593,9.330000,6.920000,1.486269e+04,1.102356e+04,3.839130e+03
2,Morocco,Clothes,Online,M,9/14/2013,667593514,10/19/2013,4611,109.279999,35.840000,5.038901e+05,1.652582e+05,3.386318e+05
3,Morocco,Clothes,Online,M,9/14/2013,667593514,10/19/2013,4611,109.279999,35.840000,5.038901e+05,1.652582e+05,3.386318e+05
4,Papua New Guinea,Meat,Offline,M,5/15/2015,940995585,6/4/2015,360,421.890015,364.690002,1.518804e+05,1.312884e+05,2.059200e+04
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1500009,India,Snacks,Online,L,10/10/2012,440306556,11/20/2012,5349,152.580002,97.440002,NaN,NaN,NaN
1500010,NaN,Household,Online,L,12/5/2016,937431466,12/8/2016,5657,668.270020,502.540009,3.780404e+06,2.842869e+06,9.375346e+05
1500011,NaN,Household,Offline,C,11/19/2011,274930989,12/13/2011,7044,668.270020,502.540009,4.707294e+06,3.539892e+06,1.167402e+06
1500012,Brunei,Cereal,Offline,L,5/28/2013,153842341,7/16/2013,4222,205.699997,117.110001,NaN,NaN,NaN


##### 7.DateTime Type

In [17]:

df["Order Date"] = pd.to_datetime(df["Order Date"])
df["Ship Date"] = pd.to_datetime(df["Ship Date"])

memory_usage = df.memory_usage(deep=True) / 1024 ** 2
memory_usage.sum()


161.5104217529297

##### 8. Outcome

In [20]:
datetime_memory = df.memory_usage(index=False, deep=True).sum() / (2 ** 30)

result_memory = round(100*(start_memory - datetime_memory)/start_memory, 2)

print(f'Reduced dataframe size: {result_memory}%')

Reduced dataframe size: 78.36%
